In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It", api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10d99eba0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10d99f770>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hello, my name is Nayz and I am a software engineer")])

AIMessage(content="Hello Nayz, it's nice to meet you!\n\nThat's awesome, I'm always interested in talking to people who work in software engineering. What kind of things do you enjoy working on?  \n\nDo you have any projects you're particularly proud of?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 22, 'total_tokens': 82, 'completion_time': 0.109090909, 'prompt_time': 0.00132378, 'queue_time': 0.251799485, 'total_time': 0.110414689}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--1d7402b1-6a51-47ba-b0d6-794303e82584-0', usage_metadata={'input_tokens': 22, 'output_tokens': 60, 'total_tokens': 82})

In [5]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi, my name is Nayz and I am a software engineer"),
        AIMessage(content="Hello Nayz, it's nice to meet you!\n\nThat's awesome, I'm always interested in talking to people who work in software engineering. What kind of things do you enjoy working on?  \n\nDo you have any projects you're particularly proud of?\n"),
        HumanMessage(content="What is my name? And what do i do?")
    ]
)

AIMessage(content="You are Nayz, and you're a software engineer!  😊  \n\nIs there anything else you'd like to tell me about yourself or your work?  I'm curious to learn more. \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 101, 'total_tokens': 149, 'completion_time': 0.087272727, 'prompt_time': 0.002725884, 'queue_time': 0.246472717, 'total_time': 0.089998611}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--bd13560b-873c-441f-a252-45ef5d0fa758-0', usage_metadata={'input_tokens': 101, 'output_tokens': 48, 'total_tokens': 149})

### Message history 

We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [10]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(
    model, 
    get_session_history
)

In [8]:
config = {
    "configurable": {
        "session_id": "chat1"
    }
}

In [12]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi, My name is Nayz and I am a Software Engineer")],
    config=config
)

In [13]:
response.content

"Hey Nayz, it's great to meet you!\n\nBeing a Software Engineer is pretty cool. What kind of projects do you usually work on?  Do you have a favorite programming language or tool you like to use?\n"

In [18]:
with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config
)

AIMessage(content='You told me your name is Nayz!  \n\nIs there something else I can help you with?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 203, 'total_tokens': 226, 'completion_time': 0.041818182, 'prompt_time': 0.004634167, 'queue_time': 0.249646137, 'total_time': 0.046452349}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--703167e8-cc3b-4d9b-974d-cfb6803e556d-0', usage_metadata={'input_tokens': 203, 'output_tokens': 23, 'total_tokens': 226})

In [16]:
another_config = {
    "configurable": {
        "session_id": "chat2"
    }
}

response2 = with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=another_config
)

In [17]:
response2.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to know! 😊\n"

### Prompt Templates

Prompt templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add more in more input besides just the messages. 

In [19]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all the questions to the best of your ability."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model | parser

In [20]:
chain.invoke({"messages": [HumanMessage(content="Hi, my name is Nayz and I am a software engineer")]})

"Hello Nayz, it's nice to meet you!\n\nAs a helpful assistant, I'm here to assist you with any questions or tasks you may have. Feel free to ask me anything, and I'll do my best to provide you with helpful and informative answers.\n\nWhat can I do for you today?\n"

In [22]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history)

In [25]:
config = {
    "configurable": {
        "session_id": "chat3"
    }
}

response = with_message_history.invoke(
    [HumanMessage(content="Hi, My name is Nayz and I am a Software Engineer")],
    config=config
)

response

"Hi Nayz! It's great to meet you.  What can I do for you today?  Are you working on any interesting projects right now? \n\nI'm excited to learn more about what you do as a Software Engineer. 😊  \n"

In [26]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer all the questions to the best of your ability in {language}"),
    MessagesPlaceholder(variable_name="messages")
])

chain = prompt | model | parser

In [27]:
response = chain.invoke(
    {
        "messages": [
            HumanMessage(content="Hi, My name is Nayz and I am a Software Engineer")
        ],
        "language": "German"
    },
    config=config
)

response

'Hallo Nayz!\n\nEs freut mich, dich kennenzulernen. Als Softwareentwickler bist du bestimmt sehr gefragt. \n\nKann ich dir bei etwas helfen?  Vielleicht hast du Fragen zu einem bestimmten Code, brauchst du Inspiration für ein neues Projekt oder möchtest einfach nur über Softwareentwicklung reden?  \n\n'

 Lets now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [29]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [30]:
config = {
    "configurable": {
        "session_id": "chat5"
    }
}

response = with_message_history.invoke(
    {
        "messages": [
            HumanMessage(content="Hi, My name is Nayz and I am a Software Engineer")
        ],
        "language": "German"
    },
    config=config
)

response

'Hallo Nayz!\n\nEs freut mich, dich kennenzulernen. Als Software-Ingenieur bin ich gespannt, wie ich dir helfen kann. \n\nWas kann ich für dich tun? \n'

In [36]:
response = with_message_history.invoke(
    {
        "messages": [
            HumanMessage(content="What do i do?")
        ],
        "language": "German"
    },
    config=config
)

response

'Software Engineer'

### Manage the conversation history 

One important concept to undertand when building chatbots is how to manager conversation history. If left unmanaged, the list of messsages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in. 

In [44]:
from langchain_core.messages import SystemMessage, trim_messages
trimmer = trim_messages(
    max_tokens=70,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="You are a good assistant"),
    HumanMessage(content="Hi! I am bob"),
    AIMessage(content="Hi !"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="Nice"),
    HumanMessage(content="What is 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="Thank you"),
    AIMessage(content="No problem"),
    HumanMessage(content="Having fun ?"),
    AIMessage(content="yes!")
]

trimmer.invoke(messages)

[SystemMessage(content='You are a good assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi! I am bob', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi !', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Thank you', additional_kwargs={}, response_metadata={}),
 AIMessage(content='No problem', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Having fun ?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [46]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(
        messages=itemgetter("messages") | trimmer
    ) | prompt | model | parser
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="What ice cream do i like")],
        "language": "German"
    }
)

response

'You like Vanilleeis. 🍦  \n\n\n\n'

In [48]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="What math problem did i ask you?")],
        "language": "German"
    }
)

response

'You asked me: "What is 2 + 2" \n\n\n'

In [53]:
## Lets wrap this in the message history

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

config = {
    "configurable": {
        "session_id": "chat6"
    }
}

response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="What math problem did i ask you?")],
        "language": "German"
    },
    config=config
)

response

'Als hilfreicher Assistent kann ich mich nicht an frühere Gespräche erinnern.\n\nKönnten Sie mir bitte das Mathematikproblem nennen, das Sie mich gerade gestellt haben? \n'